In [102]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import torch

In [103]:
alert_df=pd.read_json("/Users/fortes/neuefische/Capstone-WIMT/data/alerts.json")

In [104]:
notification_df=pd.read_json("/Users/fortes/neuefische/Capstone-WIMT/data/notifications.json")

In [105]:
corazon_df= pd.read_json("/Users/fortes/neuefische/Capstone-WIMT/data/notification-labels-to-alert-surrogate-ids.json")

In [106]:
notification_df

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value
0,20210529,1622251203899000,notification_receive,283a1,label,-Maph6wn8GA3SjPIc9HA
1,20210529,1622257988561000,notification_receive,283a1,label,-Maq63vqDUxQRawQ_3Au
2,20210529,1622258235184001,notification_receive,283a1,label,-Maq7-D7bhcEaFBU1LDW
3,20210529,1622260301025000,notification_receive,283a1,label,-MaqEsbKAqlxGG-RupjR
4,20210529,1622293962198000,notification_receive,283a1,label,-MasFHCt_lTuNt_x16dH
...,...,...,...,...,...,...
2278146,20210210,1612909178334000,notification_receive,fff18,label,0NbaiCRr6xGFd2035-Gdlw
2278147,20210210,1612969431222000,notification_receive,fff18,label,EMDjKLFr6xGRbrsa0gDjhg
2278148,20210210,1612978431552000,notification_receive,fff18,label,YE2VDMZr6xGRbrsa0gDjhg
2278149,20210210,1612985245425000,notification_receive,fff18,label,YBu-7NVr6xGRbrsa0gDjhg


In [107]:
#!pip3 install transformers --no-input

from transformers import pipeline

nlp_pos_syntax = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-pos-syntax",
    tokenizer="mrm8488/bert-spanish-cased-finetuned-pos-syntax"
)

text = 'Considera retrasos por servicios de emergencia atendiendo incendio en Mesones y Av. 20 de Noviembre.'

nlp_pos_syntax(text)[1:len(nlp_pos_syntax(text))-1]

In [108]:
cause_effect_table= pd.pivot_table(alert_df, index=['cause'], columns='effect',aggfunc='count',fill_value=0,values="is_global", margins=True, margins_name= 'Total')

In [109]:
cause_effect_table

effect,,CirculationRestored,CirculationShutdown,Delays,Evacuation,FullCapacity,HighWaitingTime,InterimService,LaneReduction,RouteDetour,SecuritySpeed,SuspensionOfService,TrafficImpact,Total
cause,,,,,,,,,,,,,,
BrokenTrafficLight,0,0,0,23,0,0,0,0,2,0,0,0,22,47
BrokenVehicle,0,0,4,390,0,0,0,0,39,0,0,12,55,500
COVID19,0,0,7,1,0,5,0,0,0,0,0,18,1,32
Counterflow,0,0,0,2,0,0,0,0,0,0,0,0,2,4
CycleRide,0,0,4,3,0,0,0,0,0,0,0,0,16,23
Demonstration,1,321,1896,166,0,0,0,65,54,5,6,251,226,2991
EmergencyServices,2,1,20,860,0,0,0,0,208,0,0,4,253,1348
Event,0,2,4,0,0,0,0,0,2,0,3,5,0,16
Explosion,0,0,0,0,0,0,0,0,0,0,0,0,1,1


Dropping the rows that contain source in join_field

In [110]:
notification_df["join_field"].value_counts()

label      2203067
source       38591
alertId      36493
Name: join_field, dtype: int64

In [111]:
source_purge= notification_df[(notification_df["join_field"]=="source")].index
notification_df.drop(source_purge, axis=0, inplace=True)

Fixing the event_name column:

In [112]:
notification_df["event_name"].replace("notification_receive","notification_received", inplace=True)

In [113]:
notification_df["event_name"].replace("notification_open","notification_opened", inplace=True)

In [114]:
merged_df= pd.merge(notification_df, alert_df, left_on="join_key_value", right_on="document_id", how="outer")

In [115]:
print (notification_df.shape)
print (alert_df.shape)
print (merged_df.shape)

(2239560, 6)
(13542, 11)
(2305578, 17)


In [116]:
corazon_df

,day,corazon_surrogate_id,notification_label_id
0,2020-12-11 06:00:00 UTC,ILXZRxI86xGr4sWpUpBiwQ,FAQZVhI86xGr4sWpUpBiwQ
1,2020-12-12 06:00:00 UTC,QPvDTqE86xGB18lnrjsyMg,cfxfVKE86xGB18lnrjsyMg
2,2020-11-05 06:00:00 UTC,MIp_iZQf6xGvT1FRo8mp4A,1P2GjJQf6xGvT1FRo8mp4A
3,2020-11-05 06:00:00 UTC,cBvxJowf6xGvT1FRo8mp4A,Q1JpKowf6xGvT1FRo8mp4A
4,2020-11-05 06:00:00 UTC,kAsi5GAf6xGGvGubJnOcaQ,sFRt6WAf6xGGvGubJnOcaQ
...,...,...,...
166614,2021-02-17 06:00:00 UTC,UMzK_DBx6xG3U-XjxJ_HhA,23k4BzFx6xG3U-XjxJ_HhA
166615,2021-02-17 06:00:00 UTC,UMzK_DBx6xG3U-XjxJ_HhA,5KA4BzFx6xG3U-XjxJ_HhA
166616,2021-02-17 06:00:00 UTC,UMzK_DBx6xG3U-XjxJ_HhA,8Hk4BzFx6xG3U-XjxJ_HhA
166617,2021-02-17 06:00:00 UTC,8J0ZPTBx6xG3U-XjxJ_HhA,siVCSjBx6xG3U-XjxJ_HhA


    IMPORTANT COMBO

In [117]:
notification_df[""].isin().value_counts()

KeyError: ''

    TESTING SHIT OUT

In [ ]:
label_merger= notification_df.query("join_field == 'label'")
alert_id_merger= notification_df.query("join_field == 'alertId'")
merged_df_1= pd.merge(label_merger, alert_df, left_on="join_key_value", right_on="document_id", how="outer")
merged_df_2= pd.merge(alert_id_merger, alert_df, left_on="join_key_value", right_on="document_id", how="outer")

In [123]:
alert_corazon = pd.merge(alert_df,corazon_df, left_on='surrogate_id', right_on='corazon_surrogate_id', how='left')

In [124]:
alert_corazon

,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates,day,corazon_surrogate_id,notification_label_id
0,-MUH7_dRaJMkhnxY9bHs,0BBrcVJ26xGCgpvcY4zoVw,2021-02-24 03:43:22+00:00,2021-02-24 03:58:39+00:00,2021-02-24 04:21:00+00:00,Fire,Delays,🔥 Considera retrasos por servicios de emergenc...,false,GtvOEQAFZ0GtU6u4AXwvPg,"[{'latitude': -99.13399344738725, 'longitude':...",2021-02-23 06:00:00 UTC,0BBrcVJ26xGCgpvcY4zoVw,sArEdVJ26xGCgpvcY4zoVw
1,-MUH7_dRaJMkhnxY9bHs,0BBrcVJ26xGCgpvcY4zoVw,2021-02-24 03:43:22+00:00,2021-02-24 03:58:39+00:00,2021-02-24 04:21:00+00:00,Fire,Delays,🔥 Considera retrasos por servicios de emergenc...,false,GtvOEQAFZ0GtU6u4AXwvPg,"[{'latitude': -99.13399344738725, 'longitude':...",2021-02-23 06:00:00 UTC,0BBrcVJ26xGCgpvcY4zoVw,oOPDdVJ26xGCgpvcY4zoVw
2,-MUH7_dRaJMkhnxY9bHs,0BBrcVJ26xGCgpvcY4zoVw,2021-02-24 03:43:22+00:00,2021-02-24 03:58:39+00:00,2021-02-24 04:21:00+00:00,Fire,Delays,🔥 Considera retrasos por servicios de emergenc...,false,JfA8Bw8Zp024Kqu4AXiSpQ,"[{'latitude': -99.13399344738725, 'longitude':...",2021-02-23 06:00:00 UTC,0BBrcVJ26xGCgpvcY4zoVw,sArEdVJ26xGCgpvcY4zoVw
3,-MUH7_dRaJMkhnxY9bHs,0BBrcVJ26xGCgpvcY4zoVw,2021-02-24 03:43:22+00:00,2021-02-24 03:58:39+00:00,2021-02-24 04:21:00+00:00,Fire,Delays,🔥 Considera retrasos por servicios de emergenc...,false,JfA8Bw8Zp024Kqu4AXiSpQ,"[{'latitude': -99.13399344738725, 'longitude':...",2021-02-23 06:00:00 UTC,0BBrcVJ26xGCgpvcY4zoVw,oOPDdVJ26xGCgpvcY4zoVw
4,-MX9U6P85DXAwGIaDsgi,kIdXsXOS6xGH7OsWJh6kOw,2021-03-31 22:51:55+00:00,2021-03-31 22:54:19+00:00,2021-04-01 00:01:01+00:00,Fire,TrafficImpact,🔥 ¡Anticípate! Afectación vial por servicios d...,false,GtvOEQAFZ0GtU6u4AXwvPg,"[{'latitude': -99.13838681335923, 'longitude':...",2021-03-31 06:00:00 UTC,kIdXsXOS6xGH7OsWJh6kOw,4J8FuXOS6xGH7OsWJh6kOw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247733,-MS3yFJ5NoOacwMyGxSs,kO_VUshg6xGXlUHhke6ddw,2021-01-27 17:51:46+00:00,2021-01-27 17:51:51+00:00,2021-01-27 19:01:00+00:00,BrokenTrafficLight,Delays,⏳ ¡Chin! Hay retrasos por semáforos descompues...,false,zCy9zG00HEqGeKu4AWZYNQ,"[{'latitude': -99.17786568559703, 'longitude':...",2021-01-27 06:00:00 UTC,kO_VUshg6xGXlUHhke6ddw,BDMQVshg6xGXlUHhke6ddw
247734,-MS3yFJ5NoOacwMyGxSs,kO_VUshg6xGXlUHhke6ddw,2021-01-27 17:51:46+00:00,2021-01-27 17:51:51+00:00,2021-01-27 19:01:00+00:00,BrokenTrafficLight,Delays,⏳ ¡Chin! Hay retrasos por semáforos descompues...,false,zCy9zG00HEqGeKu4AWZYNQ,"[{'latitude': -99.17786568559703, 'longitude':...",2021-01-27 06:00:00 UTC,kO_VUshg6xGXlUHhke6ddw,ATMQVshg6xGXlUHhke6ddw
247735,-MS3yFJ5NoOacwMyGxSs,kO_VUshg6xGXlUHhke6ddw,2021-01-27 17:51:46+00:00,2021-01-27 17:51:51+00:00,2021-01-27 19:01:00+00:00,BrokenTrafficLight,Delays,⏳ ¡Chin! Hay retrasos por semáforos descompues...,false,zCy9zG00HEqGeKu4AWZYNQ,"[{'latitude': -99.17786568559703, 'longitude':...",2021-01-27 06:00:00 UTC,kO_VUshg6xGXlUHhke6ddw,AzMQVshg6xGXlUHhke6ddw
247736,-Md1ulS_9uAaVNQEkBLN,UBnDEqjV6xGNWi2nI8j1Cg,2021-06-25 11:25:41+00:00,2021-06-25 11:25:49+00:00,2021-06-25 12:01:01+00:00,BrokenTrafficLight,Delays,⏳🚦 ¡Chin! Retrasos por semáforos descompuestos...,false,zCy9zG00HEqGeKu4AWZYNQ,"[{'latitude': -99.15550577331345, 'longitude':...",2021-06-25 05:00:00 UTC,UBnDEqjV6xGNWi2nI8j1Cg,0JSGF6jV6xGNWi2nI8j1Cg


In [ ]:
df1.merge(df2,left_on='col_name1', right_on='col_name2').merge(df3,left_on='col_name1', right_on='col_name3').drop(columns=['col_name2', 'col_name3']).rename(columns={'col_name1':'col_name'})

In [78]:
alert_df['surrogate_id'].isin(corazon_df['corazon_surrogate_id']).value_counts()

True     13448
False       94
Name: surrogate_id, dtype: int64

In [118]:
corazon_df.shape

(166619, 3)

In [84]:
notification_df['join_key_value'].isin(alert_df['document_id']).value_counts()

False    2113465
True      126095
Name: join_key_value, dtype: int64

In [101]:
notification_df['join_field'].value_counts()

label      2203067
alertId      36493
Name: join_field, dtype: int64

In [87]:
notification_df['join_key_value'].isin(corazon_df['notification_label_id']).value_counts()

True     2100518
False     139042
Name: join_key_value, dtype: int64

In [76]:
notification_df['join_key_value'].isin(alert_df['document_id']).value_counts()

False    2113465
True      126095
Name: join_key_value, dtype: int64

In [77]:
notification_df['join_key_value'].isin(alert_df['surrogate_id']).value_counts()

False    2239560
Name: join_key_value, dtype: int64

In [ ]:
merged_df.head()

TESTS FROM BEFORE

In [66]:
#label_merger.where(label_merger["join_key_value"] == alert_df["surrogate_id"],True, False)
notification_df['join_key_value'].isin(alert_df['document_id']).value_counts()

#dfAns = df1.merge(df2, on = 'id', how = 'outer').merge(df3, on='id', how='outer').fillna(0)

False    2113465
True      126095
Name: join_key_value, dtype: int64

In [67]:
notification_df['join_key_value'].isin(alert_df['surrogate_id']).value_counts()

False    2239560
Name: join_key_value, dtype: int64